# Plots of the gravitational-wave parameter space

Michael J. Williams 2023

In [ ]:
import multiprocessing as mp

import bilby
import matplotlib.pyplot as plt
import numpy as np

from thesis_utils import colours
from thesis_utils.plotting import get_default_figsize, save_figure, set_plotting
from thesis_utils.gw import injections, get_cbc_parameter_labels

set_plotting()

In [ ]:
sampling_frequency = 2048
duration = 4.0

In [ ]:
injection_parameters = injections.BBH_GW150914.bilby_format()
print(f"Injection parameters: {injection_parameters}")

In [ ]:
priors = bilby.gw.prior.BBHPriorDict()
for key in [
    "a_1",
    "a_2",
    "tilt_1",
    "tilt_2",
    "phi_12",
    "phi_jl",
    "luminosity_distance",
    "psi",
    "geocent_time",
    "ra",
    "dec",
    "theta_jn",
    "phase",
]:
    priors[key] = injection_parameters[key]

priors["chirp_mass"] = bilby.gw.prior.UniformInComponentsChirpMass(minimum=15, maximum=35)

In [ ]:
waveform_arguments = dict(
    waveform_approximant="IMRPhenomXP", reference_frequency=50.0
)

# Create the waveform_generator
waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    sampling_frequency=sampling_frequency,
    duration=duration,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=(
        bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
    ),
    waveform_arguments=waveform_arguments,
)

In [ ]:
ifos = bilby.gw.detector.InterferometerList(["H1", "L1"])
ifos.set_strain_data_from_zero_noise(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=injection_parameters["geocent_time"] - 3,
)
ifos.inject_signal(
    waveform_generator=waveform_generator,
    parameters=injection_parameters,
)

In [ ]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=ifos,
    priors=priors,
    waveform_generator=waveform_generator,
    
)

In [ ]:
parameters = injection_parameters.copy()
parameters.pop("mass_1")
parameters.pop("mass_2")

In [ ]:
n_grid = 100
grid = np.meshgrid(
    np.linspace(priors["chirp_mass"].minimum, priors["chirp_mass"].maximum, num=n_grid, endpoint=True),
    np.linspace(priors["mass_ratio"].minimum, priors["mass_ratio"].maximum, num=n_grid, endpoint=True)
)
grid_dicts = [
    dict(parameters, **dict(mass_ratio=q, chirp_mass=m)) for m, q in zip(grid[0].flatten(), grid[1].flatten())
]

In [ ]:
def log_likelihood(parameters):
    likelihood.parameters = parameters
    return likelihood.log_likelihood()

In [ ]:
log_likelihood(grid_dicts[0])

In [ ]:
with mp.Pool(8) as pool:
    log_l = np.array(pool.map(log_likelihood, grid_dicts))

In [ ]:
log_l_grid = log_l.reshape(n_grid, n_grid)

In [ ]:
injection_all_parameters = bilby.gw.conversion.generate_all_bbh_parameters(injection_parameters.copy())

In [ ]:
figsize = 0.5 * get_default_figsize()

fig = plt.figure(figsize=figsize)
plt.contourf(grid[0], grid[1], log_l_grid, cmap="cividis")
plt.colorbar(label="Log-likelihood")
plt.xlabel(get_cbc_parameter_labels("chirp_mass", units=True))
plt.ylabel(get_cbc_parameter_labels("mass_ratio", units=True))
plt.axvline(injection_all_parameters["chirp_mass"], color=colours.pillarbox)
plt.axhline(injection_all_parameters["mass_ratio"], color=colours.pillarbox)
plt.scatter(
    injection_all_parameters["chirp_mass"],
    injection_all_parameters["mass_ratio"],
    marker="s",
    color=colours.pillarbox,
)
save_figure(fig, "chirp_mass_vs_mass_ratio", "figures")
plt.show()

## Prior plots

In [ ]:
from bilby.core.prior import Uniform, Constraint
from bilby.gw.prior import CBCPriorDict

In [ ]:
component_priors = CBCPriorDict(dict(
    mass_1=Uniform(5, 100),
    mass_2=Uniform(5, 100),
    mass_ratio=Constraint(0.05, 1),
    chirp_mass=Constraint(1, 1000),
))

In [ ]:
mplot_min = 5
mplot_max = 100
m_vec = np.linspace(mplot_min, mplot_max, 500, endpoint=True)

In [ ]:
m1, m2 = np.meshgrid(m_vec, m_vec)

In [ ]:
samples = dict(mass_1=m1.flatten(), mass_2=m2.flatten())
samples["mass_ratio"] = samples["mass_2"] / samples["mass_1"]
samples["chirp_mass"] = bilby.gw.conversion.component_masses_to_chirp_mass(samples["mass_1"], samples["mass_2"])

In [ ]:
log_prior = component_priors.ln_prob(samples, axis=0).reshape(*m1.shape)
valid = np.isfinite(log_prior)

In [ ]:
q = samples["mass_ratio"].reshape(*m1.shape)
q = np.ma.array(q, mask=~valid, fill_value=np.nan)
mc = samples["chirp_mass"].reshape(*m1.shape)
mc = np.ma.array(mc, mask=~valid, fill_value=np.nan)
mtot = np.ma.array(m1 + m2, mask=~valid, fill_value=np.nan)

In [ ]:
fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)

mc_contour = axs[0].contourf(
    m1, m2, mc,
    levels=16,
    cmap="crest",
)
q_levels = np.arange(0.05, 1.05, 0.05)
q_contour = axs[1].contourf(
    m1, m2, q,
    cmap="crest",
    levels=q_levels,
)

m1_label, m2_label = get_cbc_parameter_labels(["mass_1", "mass_2"])
for ax in axs:
    ax.set(adjustable='box', aspect='equal')
    ax.set_xlabel(m1_label)
axs[0].set_ylabel(m2_label)

plt.colorbar(
    mc_contour,
    ax=axs[0],
    orientation="horizontal",
    label=get_cbc_parameter_labels("chirp_mass")
)
plt.colorbar(
    q_contour,
    ax=axs[1],
    orientation="horizontal",
    label=get_cbc_parameter_labels("mass_ratio"),
    ticks=q_levels[1::2],
)
plt.show()
save_figure(fig, "mass_space")